In [27]:
import numpy as np
import pandas as pd

import tensorflow as tf
import keras
import keras.backend as K
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score

pd.options.display.max_columns = 100

%matplotlib inline

from keras.applications import VGG16
from keras.applications import InceptionV3
from keras.applications import ResNet50
#from keras.applications import DenseNet121

K.clear_session()

from keras import models 
from keras import layers 

import os, shutil
import glob


VGG = True
ALL_LAYER_TRAIN = True
graph_dir = '20180228_2'
TEST_LABEL = 'Atelectasis'

tf_graph_dir='./tfgraph/'+graph_dir



try:
    os.makedirs(tf_graph_dir)
    print ('Dir Created!!')
except:
    os.system('rm '+tf_graph_dir+'/*')
    print ('Dir already, files removed!!')
 


Dir already, files removed!!


In [12]:


K.clear_session()

if VGG :
    IMG_SIZE = 150
    LAYER_NAME = 'block5_conv1'
    conv_base = VGG16(weights='imagenet', include_top=False, input_shape=(IMG_SIZE,IMG_SIZE,3))
else:
    IMG_SIZE = 197
    LAYER_NAME = 'res5a_branch2a'
    conv_base = ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_SIZE,IMG_SIZE,3))

conv_base.summary()




_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

In [13]:



from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers





In [14]:

conv_base.trainable = True

if ALL_LAYER_TRAIN :
    
    set_trainable = False
    for layer in conv_base.layers:
        if layer.name == LAYER_NAME:
            set_trainable = True
        if set_trainable:
            layer.trainable = True
        else:
            layer.trainable = False

            

In [28]:

_dir = '/data/private/noah/medi_image/data/binary/'
base_dir = os.path.join(_dir, TEST_LABEL)

train_dir = os.path.join(base_dir, 'train')
valid_dir = os.path.join(base_dir, 'test')
test_dir = os.path.join(base_dir, 'test')

label_name = ['Atelectasis' , 'Cardiomegaly', 'Effusion'
              , 'Infiltration', 'Mass', 'Nodule'
              , 'Pneumonia', 'Pneumothorax', 'Consolidation'
              , 'Edema', 'Emphysema', 'Fibrosis'
              , 'Pleural_Thickening', 'Hernia', 'None']

NUM_CLASSES = len(label_name)




In [29]:

## Get Number of Files 

print ('Test Labe : ', TEST_LABEL)

tmp_ts_label = os.path.join(train_dir,TEST_LABEL)
list_file = glob.glob(tmp_ts_label+'/*.png')
train_label_file_num = len(list_file)
tmp_ts_label = os.path.join(train_dir,'None')
list_file = glob.glob(tmp_ts_label+'/*.png')
train_none_file_num = len(list_file)
print ('Train Label - ', train_label_file_num, 'None - ', train_none_file_num)
print ('Train Label : %0.2f'%(100*(float)(train_none_file_num/(train_label_file_num+train_none_file_num))),"%")

tmp_ts_label = os.path.join(valid_dir,TEST_LABEL)
list_file = glob.glob(tmp_ts_label+'/*.png')
train_label_file_num = len(list_file)
tmp_ts_label = os.path.join(valid_dir,'None')
list_file = glob.glob(tmp_ts_label+'/*.png')
train_none_file_num = len(list_file)
print ('Valid Label - ', train_label_file_num, 'None - ', train_none_file_num)
print ('Valid Label : %0.2f'%(100*(float)(train_none_file_num/(train_label_file_num+train_none_file_num))),"%")

#tmp_ts_label = os.path.join(valid_dir,TEST_LABEL)
#list_file = glob.glob(tmp_ts_label+'/*.png')






Test Labe :  Atelectasis
Train Label -  8280 None -  50500
Train Label : 85.91 %
Valid Label -  3279 None -  9861
Valid Label : 75.05 %


In [6]:
with tf.device("/gpu:0"):
    
    model = models.Sequential()
    model.add()
    model.add(conv_base)
    model.add(layers.Flatten())
#    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(512,activation='relu'))
#    model.add(layers.Dense(256,activation='relu'))
#    model.add(layers.Dense(NUM_CLASSES, activation='softmax'))
    model.add(layers.Dense(1, activation='sigmoid'))
    
model.summary()



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 1, 1, 2048)        23587712  
_________________________________________________________________
flatten_1 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               1049088   
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 513       
Total params: 24,637,313
Trainable params: 16,025,601
Non-trainable params: 8,611,712
_________________________________________________________________


In [8]:

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

test_dataget = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_SIZE,IMG_SIZE),
    batch_size=16,
    class_mode='binary') #'binary')

vlaid_generator = test_dataget.flow_from_directory(
    valid_dir,
    target_size=(IMG_SIZE,IMG_SIZE),
    batch_size=16,
    class_mode='binary' ) #'binary')

with tf.device("/gpu:0"):
    model.compile(loss='binary_crossentropy', 
                optimizer = optimizers.RMSprop(lr=1e-5),
                 metrics=['accuracy'])

    tb_hist = keras.callbacks.TensorBoard(log_dir=tf_graph_dir, histogram_freq=0, write_graph=True, write_images=True)

    history = model.fit_generator(
        train_generator,
        steps_per_epoch=100,
        epochs=2,
        validation_data=vlaid_generator,
        validation_steps=50,
        callbacks=[tb_hist])



Found 64282 images belonging to 2 classes.
Found 15973 images belonging to 2 classes.
Epoch 1/2
100/100 [==============================] - 60s 603ms/step - loss: 0.5135 - acc: 0.7900 - val_loss: 0.7917 - val_acc: 0.6175
Epoch 2/2
100/100 [==============================] - 54s 540ms/step - loss: 0.5104 - acc: 0.7756 - val_loss: 0.7988 - val_acc: 0.6175


In [ ]:
model.predict(vlaid_generator)

In [19]:

vlaid_generator
